# 60分钟入门深度学习工具-PyTorch(五、数据并行(选读))
**作者**：Soumith Chintala


原文翻译自：https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html
    
中文翻译、注释制作：黄海广

github：https://github.com/fengdu78

代码全部测试通过。

配置环境：PyTorch 1.0，python 3.6，

主机：显卡：一块1080ti；内存：32g（注：绝大部分代码不需要GPU）
![公众号](images/gongzhong.jpg)
### 目录
* 1.[Pytorch是什么？](60分钟入门PyTorch-1.PyTorch是什么？.ipynb)
* 2.[AUTOGRAD](60分钟入门PyTorch-2.AUTOGRAD.ipynb)
* 3.[神经网络](60分钟入门PyTorch-3.神经网络.ipynb)
* 4.[训练一个分类器](60分钟入门PyTorch-4.训练一个分类器.ipynb)
* 5.[数据并行](60分钟入门PyTorch-5.数据并行.ipynb)

# 五、数据并行(选读)

**作者**:Sung Kim和Jenny Kang

在这个教程里,我们将学习如何使用数据并行(DataParallel)来使用多GPU。

PyTorch非常容易的就可以使用GPU,你可以用如下方式把一个模型放到GPU上:

`device = torch.device("cuda:0")`

`model.to(device)`

然后你可以复制所有的张量到GPU上:

`mytensor = my_tensor.to(device)`

请注意,只调用`mytensor.gpu()`并没有复制张量到GPU上。你需要把它赋值给一个新的张量并在GPU上使用这个张量。

在多GPU上执行前向和反向传播是自然而然的事。然而，PyTorch默认将只是用一个GPU。你可以使用`DataParallel`让模型并行运行来轻易的让你的操作在多个GPU上运行。

`model = nn.DataParallel(model)`

这是这篇教程背后的核心，我们接下来将更详细的介绍它。

## 导入和参数
导入PyTorch模块和定义参数。

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Parameters and DataLoaders
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

设备：

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 虚拟数据集
制作一个虚拟（随机）数据集，你只需实现`__getitem__`。

In [3]:
class RandomDataset(Dataset):

    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len

rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),
                         batch_size=batch_size, shuffle=True)

## 简单模型
作为演示，我们的模型只接受一个输入，执行一个线性操作，然后得到结果。然而，你能在任何模型（CNN，RNN，Capsule Net等）上使用`DataParallel`。

我们在模型内部放置了一条打印语句来检测输入和输出向量的大小。请注意批等级为0时打印的内容。

In [4]:
class Model(nn.Module):
    # Our model

    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size", input.size(),
              "output size", output.size())

        return output

## 创建一个模型和数据并行
这是本教程的核心部分。首先，我们需要创建一个模型实例和检测我们是否有多个GPU。如果我们有多个GPU，我们使用`nn.DataParallel`来包装我们的模型。然后通过`model.to(device)`把模型放到GPU上。

In [5]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  model = nn.DataParallel(model)

model.to(device)

Model(
  (fc): Linear(in_features=5, out_features=2, bias=True)
)

## 运行模型
现在我们可以看输入和输出张量的大小。

In [6]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(),
          "output_size", output.size())

	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])


## 结果
当我们对30个输入和输出进行批处理时，我们和期望的一样得到30个输入和30个输出，但是如果你有多个GPU，你得到如下的结果。

## 2个GPU

如果你有2个GPU，你将看到：

## 3个GPU：
如果你有3个GPU，你将看到：

## 8个GPU：
如果你有8个GPU，你将看到：

## 总结
`DataParallel`自动的划分数据，并将作业发送到多个GPU上的多个模型。在每个模型完成作业后，`DataParallel`收集并合并结果返回给你。

更多信息请看这里：

http://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html

本章的官方代码：
* Python：[data_parallel_tutorial.py](download/data_parallel_tutorial.py)
* Jupyter notebook:[data_parallel_tutorial.ipynb](download/data_parallel_tutorial.ipynb)